In [1]:
import numpy as np

In [3]:
m = np.loadtxt('dota_old.txt', dtype=int)
#m = m[ : 20000]
print(m)

[[238347121         0         1 ...        43        33         4]
 [238347120         1         1 ...        61         6        40]
 [238347112         1         1 ...        14        71        86]
 ...
 [237908058         1         1 ...        55        16        90]
 [237908056         1         1 ...        82        14        68]
 [237908055         1         1 ...        23        51         1]]


In [2]:
def compute_synnergy(lines):
    matches_pairs = np.zeros((121, 121))
    wins_pairs = np.zeros((121, 121)) 
    wr_pairs = np.zeros((121, 121))

    for line in lines:
        rad_wins = line[1] == 1
        dire_wins = not rad_wins
        heroes = line[3:]
        rad = heroes[:5]
        dire = heroes[5:]

        for team in [rad, dire]:
            for hero1 in team:
                for hero2 in team:
                    matches_pairs[hero1, hero2] += 1

                    if rad_wins and hero1 in rad:
                        wins_pairs[hero1, hero2] += 1 
                    elif dire_wins and hero2 in dire:
                        wins_pairs[hero1, hero2] += 1

    matches_pairs[matches_pairs == 0] = 1
    wr_pairs = wins_pairs / matches_pairs
    return wr_pairs

In [4]:
def compute_counters(lines):
    matches_counter = np.zeros((121, 121))
    wins_counter = np.zeros((121, 121)) 
    wr_counter = np.zeros((121, 121))

    for line in lines:
        rad_wins = line[1] == 1
        dire_wins = not rad_wins
        heroes = line[3:]
        rad = heroes[:5]
        dire = heroes[5:]

        for hero1 in rad:
            for hero2 in dire:
                matches_counter[hero1, hero2] += 1
                matches_counter[hero2, hero1] += 1
                if rad_wins: wins_counter[hero1, hero2] += 1
                else: wins_counter[hero2, hero1] += 1

    wins_counter[matches_counter == 0] = 1
    matches_counter[matches_counter == 0] = 1
    wr_counter = wins_counter / matches_counter
    return wr_counter

In [5]:
#np.random.shuffle(m)
factor = 0.9
train_size = int(len(m) * factor)
wr_pairs = compute_synnergy(m[: train_size])
wr_counters = compute_counters(m[ : train_size])

In [6]:
print(wr_counters[50, 63])

0.5507246376811594


In [16]:
def format_data(lines):
    #inputs = np.zeros((len(lines), 121+75))
    inputs = np.zeros((len(lines), 121))
    outputs = np.zeros((len(lines)))


    for i in range(len(lines)):
        line = lines[i]
        input = inputs[i]
        outputs[i] = line[1]
        game_type = line[2]
        
        heroes = line[3:]
        rad = heroes[:5]
        dire = heroes[5:]
            
        for hero in rad:
            input[hero] = 1

        for hero in dire:
            input[hero] = -1
            
#         counter = 121
        
#         for team in [rad,dire]:
#             for hero1 in team:
#                 for hero2 in team:
#                     input[counter] = wr_pairs[hero1, hero2]
#                     counter += 1
                    
#         for hero1 in rad:
#             for hero2 in dire:
#                 input[counter] = wr_counters[hero1, hero2]
#                 counter += 1

    return (inputs, outputs)

In [17]:
inputs, outputs = format_data(m)
print(outputs)
print(inputs[10])

[0. 1. 1. ... 1. 1. 1.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.
  0. -1. -1. -1.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.
  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0. -1.  0.  0.  0.
  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


In [ ]:
print(len(inputs[0]))

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD, Adagrad, Adam
from keras import regularizers

model = Sequential()

limit = len(inputs)
train_size = int(limit * 0.8)
test_size = limit - train_size
train_in = inputs[ : train_size]
train_out = outputs[: train_size]
test_in = inputs[train_size : limit]
test_out = outputs[train_size : limit]

model.add(Dense(100, activation='relu', input_dim=len(inputs[0])))
model.add(Dropout(0.5))
# model.add(Dense(100, activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

sgd = SGD(lr=0.05)
model.compile(loss='binary_crossentropy',
              #optimizer=Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              optimizer='adadelta',
              #optimizer=sgd,
              metrics=['binary_accuracy'])

model.fit(train_in, train_out,
          epochs=20,
          validation_data=(test_in, test_out))

result = model.evaluate(test_in, test_out)

print(result[1])

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=1000, max_features=11, min_samples_split=2, max_depth=None, n_jobs=8)

In [18]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()

In [ ]:
from sklearn import neighbors

clf = neighbors.KNeighborsClassifier(5, weights='uniform')
clf = clf.fit(data['train']['in'], data['train']['out'])

In [ ]:
print()
# from sklearn import svm
# clf = svm.SVC()
# clf.fit(data['train']['in'], data['train']['out'])  

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators=10000)

In [ ]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=2000, learning_rate=1.0, max_depth=1, random_state=0)

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

clf = BaggingClassifier(LogisticRegression(),
                             max_samples=0.5, max_features=0.5)

In [ ]:
from sklearn.ensemble import VotingClassifier

clf = VotingClassifier(estimators=[('for', clf_forest), ('naive', clf_naive), ('log', clf_log)], voting='soft', weights=[1,2,2])

In [ ]:
from xgboost import XGBClassifier

clf = XGBClassifier()

In [ ]:
from sklearn.linear_model import LogisticRegressionCV

clf = LogisticRegressionCV(Cs=10, solver='saga', cv=3, max_iter=1000, n_jobs=8, verbose=0, refit=True, intercept_scaling=1.0)

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Set the parameters by cross-validation
# tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
#                      'C': [1, 10, 100, 1000]},
#                     {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

tuned_parameters = [{'n_estimators': [10, 50, 100, 200], 'max_features': [5, 11, 20, 40, 80, 121]}]

scores = ['precision']

X_train, X_test, y_train, y_test = train_test_split(inputs, outputs, test_size=0.5, random_state=0)


clf = GridSearchCV(RandomForestClassifier(), tuned_parameters, cv=5,
                       scoring='%s_macro' % 'precision')
clf.fit(X_train, y_train)

print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
print()

print("Detailed classification report:")
print()
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()
y_true, y_pred = y_test, clf.predict(X_test)
print(classification_report(y_true, y_pred))
print()


In [19]:
# from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

# print(inputs)
# print(outputs)
# limit = len(inputs)
# scores = cross_val_score(clf, inputs, outputs, cv=5)

train_size = int(len(inputs) * 0.9)
test_size = len(inputs) - train_size



clf = clf.fit(inputs[:train_size], outputs[:train_size])
preds = clf.predict(inputs[train_size:])
correct = outputs[train_size:]

# print(scores)

print(accuracy_score(correct, preds))

0.5959747671973565


In [ ]:
import pandas as pd
import numpy as np

heroes = pd.read_csv('dota-2-match-prediction-master/data/MatchOverviewTraining.csv').values
df_extra = pd.read_csv('dota-2-match-prediction-master/data/MatchDetail.csv')

In [ ]:
print(heroes)

In [ ]:
def gen_data(heroes, df_extra):
    matches = np.zeros((10000, 121), dtype=int)
    out = np.zeros((10000), dtype=int)
    
    for (i, line) in enumerate(heroes):
        out[i] = 1 if line[-1] else 0
        match_id = line[0]
        heroes = line[1:]
        rad = heroes[:5]
        dire = heroes[5:-1]
        
        for hero in rad:
            matches[i][hero] = 1
            
        for hero in dire:
            matches[i][hero] = -1
            
        
#         counter = 121
        
#         exp_gold = df_extra.loc[df_extra['match_id'] == match_id].values
#         for (j, data) in enumerate(exp_gold):
#             matches[i][counter + 2*j] = data[4]
#             matches[i][counter + 2*j + 1] = data[14]
            
    return matches, out
        
        
        

In [ ]:
input, output = gen_data(heroes, df_extra)

In [ ]:
print(input[0])

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD, Adagrad, Adam
from keras import regularizers

model = Sequential()

train_size = int(len(input) * 0.9)
test_size = len(input) - train_size
train_in = input[ : train_size]
train_out = output[: train_size]
test_in = input[train_size : ]
test_out = output[train_size : ]

model.add(Dense(100, activation='relu', input_dim=len(input[0])))
#model.add(Dropout(0.1))
# model.add(Dense(100, activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(10, activation='relu'))
#model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))

#sgd = SGD(lr=0.05)
model.compile(loss='binary_crossentropy',
              #optimizer=Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              optimizer='adadelta',
              #optimizer=sgd,
              metrics=['binary_accuracy'])

model.fit(train_in, train_out,
          epochs=200,
          validation_data=(test_in, test_out))

result = model.evaluate(test_in, test_out)

print(result[1])

In [ ]:
from xgboost import XGBClassifier

clf = XGBClassifier()

In [ ]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()

In [ ]:
#print(input[0])
clf.predict_proba([input[1]])

In [ ]:
#from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

# print(inputs)
# print(outputs)
#limit = len(inputs)
#scores = cross_val_score(clf, inputs[:limit], outputs[:limit], cv=2)

factor = 0.9
train_size = int(len(input) * factor)
test_size = len(input) - train_size
train_in, train_out, test_in, test_out = input[:train_size], output[:train_size], input[train_size:], output[train_size:]

# print(train_out, test_out)

clf = clf.fit(train_in, train_out)
preds = clf.predict(test_in)
correct = test_out

# print(scores)

print(accuracy_score(correct, preds))